In [31]:
import pandas as pd
import requests
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
oe=OrdinalEncoder()
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()

In [9]:
url="https://www.saude.pr.gov.br/sites/default/arquivos_restritos/files/documento/2020-11/INFORME_EPIDEMIOLOGICO_24_11_Geral.csv"
req = requests.get(url)
url_content = req.content
csv_file = open('covid.csv', 'wb')

csv_file.write(url_content)
csv_file.close()

In [12]:
covid_df=pd.read_csv('covid.csv', sep=';')
covid_df.head()

/Users/julio.j4pa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Recuperado    190570
recuperado         2
Name: STATUS, dtype: int64

In [28]:
covid_df.SEXO.value_counts()
#covid_df[(covid_df['OBITO']=='Sim')]

F    136656
M    124362
Name: SEXO, dtype: int64

In [64]:
#Provavelmente pessoas com covid q ainda nao recuperaram nem morreram
covid_df[(covid_df['OBITO'].isnull()) & (covid_df['STATUS'].isnull())]

,IBGE_RES_PR,IBGE_ATEND_PR,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_CONFIRMACAO_DIVULGACAO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,FEMININO,MASCULINO
167156,4106704,4118402,65,CRUZEIRO DO SUL,PARANAVAÍ,CLINICO EPIDEMIOLÓGICO,16/09/2020,17/09/2020,NaN,NaN,04/09/2020,17/09/2020,NaN,NaN,NaN,1.0,0.0


In [65]:
#Binarizaçao
tempSexo=ohe.fit_transform(covid_df[['SEXO']])
ohe.categories_
covidSexo_df=pd.DataFrame(tempSexo.toarray(), columns=['FEMININO','MASCULINO'])
novo_df=pd.merge(covid_df,covidSexo_df, left_index=True, right_index=True)
novo_df=novo_df.drop(['SEXO'], axis=1)

In [77]:
#Confrimar nao obitos
novo_df.loc[novo_df['OBITO'].isnull() & novo_df['DATA_OBITO'].isnull(), 'OBITO']='Não'
#Normalizar String

novo_df.loc[(novo_df['OBITO']=='SIM') | (novo_df['DATA_OBITO'].notnull()), 'OBITO']='Sim'

In [78]:
oe.fit_transform(novo_df[['OBITO']])
oe.categories_
novo_df.OBITO=oe.fit_transform(novo_df[['OBITO']])

In [80]:
novo_df.head()

,IBGE_RES_PR,IBGE_ATEND_PR,IDADE_ORIGINAL,MUN_RESIDENCIA,MUN_ATENDIMENTO,LABORATORIO,DATA_DIAGNOSTICO,DATA_CONFIRMACAO_DIVULGACAO,DATA_INICIO_SINTOMAS,OBITO,DATA_OBITO,DATA_OBITO_DIVULGACAO,STATUS,DATA_RECUPERADO_DIVULGACAO,FONTE_DADO_RECUPERADO,FEMININO,MASCULINO
0,4114609,4114609,47,MARECHAL CANDIDO RONDON,MARECHAL CANDIDO RONDON,"COVID-19, Biologia Molecular",10/10/2020,13/10/2020,04/10/2020,0.0,NaN,NaN,Recuperado,21/10/2020,NOTIFICA COVID,0.0,1.0
1,4125407,4125407,73,SAO JOSE DA BOA VISTA,SAO JOSE DA BOA VISTA,"COVID-19, Biologia Molecular",11/10/2020,13/10/2020,30/09/2020,0.0,NaN,NaN,Recuperado,20/10/2020,NOTIFICA COVID,0.0,1.0
2,4118204,4118204,45,PARANAGUA,PARANAGUA,"COVID-19, Biologia Molecular",09/10/2020,13/10/2020,01/10/2020,0.0,NaN,NaN,Recuperado,21/10/2020,NOTIFICA COVID,0.0,1.0
3,4102208,4102208,40,ATALAIA,ATALAIA,"COVID-19, Biologia Molecular",11/10/2020,13/10/2020,29/09/2020,0.0,NaN,NaN,Recuperado,22/10/2020,NOTIFICA COVID,1.0,0.0
4,4103602,4103602,43,CAMBARA,CAMBARA,"COVID-19, Biologia Molecular",11/10/2020,13/10/2020,05/10/2020,0.0,NaN,NaN,Recuperado,03/11/2020,NOTIFICA COVID,1.0,0.0
